This ledger contains all updates to the project. It includes important code snippets, information, explantion, references and other relavant information for all collaborators. 

# Date: 11/17/2019

Inital setup
* Created ledger
* Created git repository
* Shifted ledger to git 

Added nntools.py to folder src
* Use to inherit Experiment and other class funtionalities

Created a new folder: local_tools
* This folder contains all the custom libraries and classes that we write
* Shifted nntools.py to this folder

Created a new folder: local_notebooks
* This folder used to create different local python notebooks for each colaborators
* Any changes to functions can be added here 

### Command to download a file (e.g. a model) from Google Drive

wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=FILEID' -O FILENAME

E.g: wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1b-_sjq1Pe_dVxt5SeFmoadMfiPTPZqpz' -O ctdet_coco_resdcn18.pth

### Make external
~/MoDL_CenterNet/src/lib/external$ make

### DCNv2 for Pytorch 1.0
Repo: https://github.com/CharlesShang/DCNv2 (clone in MoDL_CenterNet/src/lib/models/networks/DCNv2/)

python setup.py build  # build

PYTHONPATH="${PYTHONPATH}:path_until_here/MoDL_CenterNet/src/lib/models/networks/DCNv2"

export PYTHONPATH

python setup.py develop --instal-dir path_until_here/MoDL_CenterNet/src/lib/models/networks/DCNv2 #install

### For Test script error on unable to find model
Copy the model to MoDL_CenterNet/exp/ctdet/model_name and rename to 'model_last.pth

# Date: 11/23/2019

## Added a local notebook to read data set

* Creating custom object opt with desired parameters just sufficient to call the class COCO
* This class can be used to execute the cocoapi toolkit to manipulate the images on the Jupyter notebook 

## Parts of the local notebook

This describes the different parts of the notebook that have been added. The goal is to run the same script in the library without the need for command line arguments. This also gives us complete control over the parameters of interest and will be benefitial for future modifications of the network. 

The empty object opt is created and then each individual feature of this object is updated based on our requirement. 

### Updating opt

The original opt object is initialized using the command line argument, when calling the main script. The aim here is to pass all these parameters to the object through the notebook. 
* opt is initialized as an empty object
* All experimental parameters, loss, data and image parameters
* The different directories for storing the model, epochs and data is stored in this object
* This can be used for any further modifications to the program 

### Loading the dataset

The dataset used is the MS COCO 2017 dataset. The data is loaded in the user directory root folder. 

The dataset can be manipulate using the cocoapi libraries.  

### Modifying the dataset

For initial testing we are modifying the dataset to a smaller size. The following code updates the json file list to use a smalller subset of the data. 

In [ ]:
N=256
numsets=1
for iter in range(numsets):
    imgIds_perm=np.random.permutation(len(all_Ids))
    tmp=imgIds_perm[0:N].astype(int)
    tmp2=[all_Ids[t] for t in tmp]
    dataset.images=tmp2
    dataset.num_samples=len(dataset.images)
    sub_inst_cat=np.zeros(90)
    for j in range(N):
        sub_cat_lab=[]
        img = dataset.coco.loadImgs(dataset.images[j])[0]
        f_name=img_dir
        f_name+=img['file_name']
        annIds = dataset.coco.getAnnIds(imgIds=img['id'])
        anns = dataset.coco.loadAnns(annIds)
        sub_cat_lab=[k['category_id'] for k in anns]
        for jj in range(90):
            t=np.where(np.asarray(sub_cat_lab)==jj)
            sub_inst_cat[jj-1]+=t[0].shape[0]
    prob_sub=(sub_inst_cat+1)/np.sum(sub_inst_cat+1)    
    plt.plot(sub_inst_cat/(np.sum(sub_inst_cat)))

Based on this script, it is possible to proivide the statistics of the smaller dataset. 

### Model

The given framework is decided to acoomodate different achitectures for object classification. The models relevant for the COCO dataset are: 
* Renet18
* Resnet 101
* DLA
* Hourglass

The original literature/libraries provide separate python scripts for loasing these models. For the inital testing and validation the Resnet 18 architechture was used. 

### Trainer

The trainer object type contains the structure for training different architectures with different loss functions. There are 2 major modules used here, for the task of object classification: 
* BaseTrainer
    * This class contains the stensil functionality for the loss, optimizer and model characteristics, not specifying them 
    * The forward and training functionality is defined by this class, executed during each epoch
    * The individual trainers are given specifc to each class
* CtdetTrainer
    * This inherits the main base trainer functionality and defines the losses specific to object detection and classification
    * In addition this module also contains the debugger for the trainer 

### Running the main loop

The initial loop test was run on a subset of the data, with a single epoch. 